In [1]:
%cd last/

/home/changhyun/workspace/Bigdata_Competition/last


In [3]:
!python preprocess.py

100%|████████████████████████████████| 730290/730290 [00:22<00:00, 33177.44it/s]


In [4]:
import tensorflow as tf

class SessionDataset:

    def __init__(self, df):

        self.df = df.sort_values(by = ['session', 'timestamp']).reset_index(drop = True) # session (int) | timestamp (int) | item (string)
        self.offsets = np.concatenate((np.zeros(1, dtype = np.int32), self.df.groupby('session').size().cumsum().values)) # indices in df where the sessions start
        self.n_sessions = len(self.offsets) - 1

        self.item_to_id = {item : i for i, item in enumerate(self.df.item.unique())}

        self.n_items = len(self.item_to_id)

    def item_to_one_hot(self, item):

        return tf.one_hot(self.item_to_id[item], depth = self.n_items)

    def extract_session(self, i, one_hot_encoded = True):

        session = self.df[self.offsets[i]:self.offsets[i+1]].copy()
        if one_hot_encoded:
            session.loc[:, 'item'] = session.item.apply(lambda x : self.item_to_one_hot(x))
        return session.item.values.tolist()

In [5]:
# y_true = (BATCH_SIZE, n_classes)   one-hot representations of the target items (ground truths)
# y_pred = (BATCH_SIZE, n_classes)   model output = next item scores (logits) for each item in the batch

sampling = False

if sampling: # = the negative items considered in the loss computation are those within the same batch
    
    def BPR(y_true, y_pred):
        to_lookup = tf.argmax(y_true, axis = 1)   # = indices of the target items
        scores = tf.nn.embedding_lookup(tf.transpose(y_pred), to_lookup)  # embedding_lookup is the same as "extract_rows". In this way, the positive items end up on the diagonal
        return tf.reduce_mean(-tf.math.log(tf.nn.sigmoid(tf.linalg.diag_part(scores) - scores)))

    def TOP1(y_true, y_pred):
        to_lookup = tf.argmax(y_true, axis = 1)
        scores = tf.nn.embedding_lookup(tf.transpose(y_pred), to_lookup)
        diag_scores = tf.linalg.diag_part(scores)
        loss_by_sample  = tf.reduce_mean(tf.nn.sigmoid(scores - diag_scores) + tf.nn.sigmoid(tf.square(scores)), axis = 0)
        loss_by_sample -= tf.nn.sigmoid(tf.square(diag_scores)) / tf.reduce_sum(tf.ones_like(diag_scores)) # only sigmoids of squares of negative items had to be added: remove those of positive items
        return tf.reduce_mean(loss_by_sample)

else: # = consider all negative items in the loss computation (only makes sense if the number of items is small, like the same order as the batch size)

    def BPR(y_true, y_pred):  # both inputs have shape (BATCH_SIZE, n_classes)
        _y_pred = tf.expand_dims(y_pred, axis = -1)  # (BATCH_SIZE, n_classes, 1) 
        mat = tf.matmul(tf.expand_dims(tf.ones_like(y_true), -1), tf.expand_dims(y_true, axis = 1)) # (BATCH_SIZE, n_classes, 1) x (BATCH_SIZE, 1, n_classes) = (BATCH_SIZE, n_classes, n_classes)
        score_diffs = tf.matmul(mat, _y_pred) # (BATCH_SIZE, n_classes, n_classes) x (BATCH_SIZE, n_classes, 1) = (BATCH_SIZE, n_classes, 1)
        score_diffs = tf.squeeze(score_diffs - _y_pred, -1) # (BATCH_SIZE, n_classes)
        return -tf.reduce_sum(tf.math.log(tf.nn.sigmoid(score_diffs)))

    def TOP1(y_true, y_pred):
        _y_pred = tf.expand_dims(y_pred, axis = -1)  # (BATCH_SIZE, n_classes) ---> (BATCH_SIZE, n_classes, 1) 
        mat = tf.matmul(tf.expand_dims(tf.ones_like(y_true), -1), tf.expand_dims(y_true, axis = 1)) # (BATCH_SIZE, n_classes, 1) x (BATCH_SIZE, 1, n_classes) --> (BATCH_SIZE, n_classes, n_classes)
        score_diffs = tf.matmul(mat, _y_pred) # (BATCH_SIZE, n_classes, n_classes) x (BATCH_SIZE, n_classes, 1) --> (BATCH_SIZE, n_classes, 1)
        score_diffs = tf.squeeze(score_diffs - _y_pred, -1) # (BATCH_SIZE, n_classes)
        loss_by_sample = tf.reduce_sum(tf.nn.sigmoid(tf.square(y_pred)), axis = -1) + \
                          tf.reduce_sum(tf.sigmoid(-score_diffs), axis = -1) + \
                        -tf.squeeze(tf.squeeze(tf.nn.sigmoid(tf.square(tf.matmul(tf.expand_dims(y_true, 1), _y_pred))), -1), -1)
        return tf.reduce_sum(loss_by_sample)

In [6]:
class Gru4Rec:

    def __init__(self, n_classes, n_layers = 1, n_hidden = 64, loss = TOP1, batch_size = 8):

        self.n_classes  = n_classes   # = number of items

        self.n_layers = n_layers  # number of stacked GRU layers
        self.n_hidden = n_hidden  # dimension of GRU cell's hidden state
        self.loss     = loss
        self.batch_size = batch_size

        self.model = self.build_model()

    def build_model(self):

        model = tf.keras.models.Sequential()
        for i in range(self.n_layers):
            model.add(tf.keras.layers.GRU(name = 'GRU_{}'.format(i+1),
                                          units      = self.n_hidden, 
                                          activation = 'relu', 
                                          stateful   = True,
                                          return_sequences = (i < self.n_layers - 1)))
        model.add(tf.keras.layers.Dense(units = self.n_classes, activation = 'linear'))   # class logits

        # track top 3 accuracy (= how often the true item is among the top 3 recommended)
        top3accuracy = lambda y_true, y_pred: tf.keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k = 3)
        top3accuracy.__name__ = 'top3_accuracy'
        model.compile(loss = self.loss, optimizer = 'adam', metrics = ['accuracy', top3accuracy])

        model.build(input_shape = (self.batch_size, 1, self.n_classes))
        print(model.summary())

        return model

    def _reset_hidden(self, i):

        for nl, layer in enumerate(self.model.layers):   # session has changed: reset corresponding hidden state
            if self._is_GRU_layer(layer) and layer.states[0] is not None:
                hidden_updated = layer.states[0].numpy()
                hidden_updated[i, :] = 0.
                self.model.layers[nl].reset_states(hidden_updated)

    def _is_GRU_layer(self, layer):

        return layer.name.startswith('GRU_')

    def train_batch_generator(self, dataset):  # session | item | timestamp
        # generates batches of training data X, y = session item, next session item

        assert dataset.n_sessions > self.batch_size, "Training set is too small. Reduce batch size or collect more training data"
        ixs = np.arange(dataset.n_sessions)

        stacks = [[]] * self.batch_size   # stacks containing batch_size REVERSED (pieces of) sessions at once. Will be emptied progressively
        next_session_id = 0

        X, y = np.empty(shape = (self.batch_size, 1, self.n_classes)), np.empty(shape = (self.batch_size, self.n_classes))    
        while True:
            X[:], y[:] = None, None
            for i in range(self.batch_size): # fill in X, y (current batch)
                # 1. If stack i is empty (only happens at first round) or has only one element: fill it with a new session
                if len(stacks[i]) <= 1:
                    if next_session_id >= dataset.n_sessions: # no more sessions available: shuffle sessions and restart
                        np.random.shuffle(ixs)
                        next_session_id = 0
                    while not len(stacks[i]) >= 2:   # ignore sessions with only one element (cannot contribute to the training)
                        stacks[i] = dataset.extract_session(ixs[next_session_id])[::-1]  # the data does not have to be all in memory at the same time: we could e.g. load a session at once
                        next_session_id += 1
                    self._reset_hidden(i)   # if session changes, the corresponding hidden state must be reset
                # 2. Stack i is now valid: set input + target variables
                X[i, 0] = stacks[i].pop()
                y[i]    = stacks[i][-1]

            yield tf.constant(X, dtype = tf.float32), tf.constant(y, dtype = tf.float32)

    def fit(self, dataset, steps_per_epoch = 10000, epochs = 5):

        checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = "gru-chkpt-{epoch:02d}.hdf5")
        self.model.fit_generator(generator       = self.train_batch_generator(dataset), 
                                 steps_per_epoch = steps_per_epoch, 
                                 epochs          = epochs,
                                 callbacks       = [checkpoint], 
                                 shuffle         = False)

In [7]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('../../dataset/big_comp/convert_time_df.csv', sep='\t')

columns = ['session']
for i in columns:
    globals()[f'encoder_{i}'] = LabelEncoder()
    df[i] = globals()[f'encoder_{i}'].fit_transform(df[i]) 

df = df.sort_values(by = ['session', 'timestamp']).reset_index(drop = True)

offsets = np.concatenate((np.zeros(1, dtype = np.int32), df.groupby('session').size().cumsum().values))

dataset_train = SessionDataset(df.iloc[~df.index.isin(offsets[1:] - 1)])  # training set: remove last element from each session

# Test set: x = penultimate item in each session, y = last item in each session
X_test = df.iloc[offsets[1:] - 2][['session', 'item']].sort_values('session').reset_index(drop = True)
y_test = df.iloc[offsets[1:] - 1][['session', 'item']].sort_values('session').reset_index(drop = True)

print("X_test")
print(X_test.head())
print('')
print("y_test")
print(y_test.head())

X_test
   session      item
0        0        완구
1        1      패션잡화
2        2        과자
3        3        과자
4        4  화장품/뷰티케어

y_test
   session     item
0        0       완구
1        1     여성의류
2        2       음료
3        3      축산물
4        4  테넌트/음식점


In [8]:
g4r = Gru4Rec(n_classes = dataset_train.n_items)
g4r.fit(dataset_train)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 GRU_1 (GRU)                 (8, 64)                   24192     
                                                                 
 dense (Dense)               (8, 60)                   3900      
                                                                 
Total params: 28,092
Trainable params: 28,092
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5


2022-08-07 03:52:17.484069: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-07 03:52:17.512622: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-07 03:52:17.512775: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-07 03:52:17.513290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

10000/10000 [==============================] - 31s 3ms/step - loss: 385.1189 - accuracy: 0.1716 - top3accuracy: 0.3500
Epoch 2/5
10000/10000 [==============================] - 30s 3ms/step - loss: 365.2920 - accuracy: 0.2436 - top3accuracy: 0.4309
Epoch 3/5
10000/10000 [==============================] - 30s 3ms/step - loss: 360.7752 - accuracy: 0.2596 - top3accuracy: 0.4422
Epoch 4/5
10000/10000 [==============================] - 30s 3ms/step - loss: 359.2264 - accuracy: 0.2689 - top3accuracy: 0.4609
Epoch 5/5
10000/10000 [==============================] - 30s 3ms/step - loss: 360.1653 - accuracy: 0.2697 - top3accuracy: 0.4520


In [25]:
final_states = np.empty(shape = (dataset_train.n_sessions, g4r.n_layers, g4r.n_hidden)) # final states will be stored here
final_states[:] = None
done = [False] * dataset_train.n_sessions   # keep track of the sessions for which the last state has already been calculated

stacks = [dataset_train.extract_session(i)[::-1] for i in range(g4r.batch_size)]
next_session_id = g4r.batch_size
batch_idx_to_session = np.arange(g4r.batch_size)   # keep track of which session is in each batch element
X = np.empty(shape = (g4r.batch_size, 1, g4r.n_classes))

g4r.model.reset_states()    # all hidden states set to 0 (starting point)

n_done = 0

In [35]:
dataset_train.n_sessions

25884

In [ ]:
from tqdm import tqdm

final_states = np.empty(shape = (dataset_train.n_sessions, g4r.n_layers, g4r.n_hidden)) # final states will be stored here
final_states[:] = None
done = [False] * dataset_train.n_sessions   # keep track of the sessions for which the last state has already been calculated

stacks = [dataset_train.extract_session(i)[::-1] for i in range(g4r.batch_size)]
next_session_id = g4r.batch_size
batch_idx_to_session = np.arange(g4r.batch_size)   # keep track of which session is in each batch element
X = np.empty(shape = (g4r.batch_size, 1, g4r.n_classes))

g4r.model.reset_states()    # all hidden states set to 0 (starting point)

n_done = 0

while n_done < dataset_train.n_sessions:
    for i in range(g4r.batch_size):
        while len(stacks[i]) == 1:  # stack i is at the end
            if not done[batch_idx_to_session[i]]:
                # save final hidden state
                final_states[batch_idx_to_session[i], :] = np.array([layer.states[0][i, :] for layer in g4r.model.layers if g4r._is_GRU_layer(layer)])
                done[batch_idx_to_session[i]] = True
                n_done += 1
                print(str(round(n_done/25884*100,1))+"% is completed.")
#                 if n_done % 100 == 0:
#                     print(f"Progress: {n_done} / {dataset_train.n_sessions}")
            if next_session_id >= dataset_train.n_sessions: # restart from the beginning (just to reach required batch size)
                next_session_id = 0
            stacks[i] = dataset_train.extract_session(next_session_id)[::-1]
            batch_idx_to_session[i] = next_session_id
            next_session_id += 1
            g4r._reset_hidden(i)   # session has changed --> reset corresponding hidden state
        X[i, 0] = stacks[i].pop()

    _ = g4r.model.predict(X)   # hidden states get updated when "predict" is called

print("All final hidden states calculated")
np.save('../../dataset/big_comp/final_states.npy', final_states, allow_pickle = False)

In [ ]:
final_states = np.load('../../dataset/big_comp/final_states.npy')

g4r.model.reset_states()

rem = dataset_train.n_sessions % g4r.batch_size
if rem > 0:
    X_test = pd.concat((X_test, X_test[:(g4r.batch_size - rem)]), axis = 0)

# Calculate next item predictions for all sessions
y_pred = np.empty(shape = (dataset_train.n_sessions, g4r.n_classes))
y_pred[:] = None
X = np.empty(shape = (g4r.batch_size, 1, g4r.n_classes))
for batch_id in range(dataset_train.n_sessions // g4r.batch_size):
    # X contains the penultimate item in the session (= last item in the training set)
    X[:] = None
    for i in range(g4r.batch_size):
        X[i, :] = dataset_train.item_to_one_hot(X_test.iloc[batch_id * g4r.batch_size + i]['item'])
    # set hidden states equal to final hidden states for sessions in the batch
    nlg = 0
    for nl, layer in enumerate(g4r.model.layers):
        if g4r._is_GRU_layer(layer):
            g4r.model.layers[nl].reset_states(final_states[batch_id * g4r.batch_size : (batch_id + 1) * g4r.batch_size, nlg, :])
            nlg += 1
    # objective: predict last element in the session
    y_pred[batch_id * g4r.batch_size : (batch_id + 1) * g4r.batch_size, :] = g4r.model.predict(X)[:g4r.batch_size]

y_pred = tf.constant(y_pred[:dataset_train.n_sessions], dtype = tf.float32)


In [44]:
data_f = pd.DataFrame(y_pred)
data_f

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,-0.101400,5.122470,0.411462,-0.236007,-0.024931,-0.231721,5.983501,0.489373,0.210659,0.313644,...,-0.013549,0.194592,-0.138011,-0.050456,-0.198736,-0.213913,-0.187095,-0.094436,-0.199338,-0.197980
1,6.134058,1.717681,-0.343599,0.297462,6.539805,-0.081693,4.824500,0.022874,0.056690,-0.091561,...,-0.179814,-0.210738,-0.313608,-0.103755,-0.234412,-0.214502,-0.186226,-0.194537,-0.225704,-0.222353
2,0.108491,9.877229,-0.307102,0.014491,0.023762,-0.102377,12.490325,0.011585,0.356762,0.051139,...,-0.092662,-0.006177,-0.102056,-0.141185,-0.150112,-0.049161,-0.328302,-0.163079,-0.139322,-0.142054
3,-0.097103,7.277634,0.665289,-0.202129,0.387593,0.094024,11.349756,1.859478,3.956362,-0.248531,...,-0.166493,-0.066305,-0.053265,-0.210834,-0.063536,-0.160048,-0.087788,-0.057035,-0.149206,-0.148674
4,-0.026675,1.723083,-0.112334,-0.112589,0.352426,-0.153636,2.945801,-0.284290,0.030442,0.023661,...,-0.191590,-0.251919,-0.098760,-0.208515,-0.168009,-0.134835,-0.077329,-0.169855,-0.120925,-0.121390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25879,-0.244383,4.529177,2.099071,-0.023050,-0.196272,-0.165463,3.440201,0.716204,1.595793,0.310500,...,-0.342207,-0.247778,-0.122210,-0.209082,-0.136593,-0.066616,-0.083899,-0.005069,-0.099946,-0.089550
25880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
data_f2 = pd.DataFrame(y_true)
data_f2

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25882,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
y_test

,session,item
0,0,완구
1,1,여성의류
2,2,음료
3,3,축산물
4,4,테넌트/음식점
...,...,...
26912,26912,담배
26913,26913,과자
26914,26914,커피/차
26915,26915,조미료


In [39]:
# Retrieve ground truths
y_true = np.empty(shape = (dataset_train.n_sessions, dataset_train.n_items))
for i in range(y_true.shape[0]):
    y_true[i, :] = dataset_train.item_to_one_hot(y_test.item.values[i])
y_true = tf.constant(y_true, dtype = tf.float32)

In [41]:
acc       = (tf.reduce_sum(tf.keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k = 1)) / y_true.shape[0]).numpy()
top_3_acc = (tf.reduce_sum(tf.keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k = 3)) / y_true.shape[0]).numpy()

print("Accuracy = {}".format(acc))
print("Top-3 accuracy = {}".format(top_3_acc))


Accuracy = 0.24860918521881104
Top-3 accuracy = 0.3709627687931061


In [47]:
TOP1()

<tf.Tensor: shape=(25884,), dtype=float32, numpy=array([1., 1., 1., ..., 0., 0., 0.], dtype=float32)>